In [109]:
import itertools
import torch
import torch.optim as optim
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import random
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA

In [110]:
import sys
root = '../../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def CreateDataset(X, y, batch_size, train = 1):

    if train == 1:
        #Create X_train_tensor
        X_train_tensor = torch.tensor(X)
        X_size = X_train_tensor.shape[0]

        #Create Y_train_tensor
        y_train_tensor = torch.tensor(y, dtype=torch.long)
        

        #Create indices
        indices_train = torch.arange(len(X_train_tensor))
        dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return train_loader, X_size

    elif train == 2:
        
        #Create X_train_tensor
        X_test_tensor = torch.tensor(X)

        #Create Y_train_tensor
        y_test_tensor = torch.tensor(y, dtype=torch.long)
        

        #Create indices
        indices_test = torch.arange(len(X_test_tensor))
        dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return test_loader, X_size

In [ ]:
train_loader = CreateDataset(batch_size = 128, train = 1, HPO=True)
#test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)
validation_loader = CreateDataset(batch_size = 128, train = 2, HPO = True)

torch.Size([5002, 4096])
torch.Size([5002, 21])
torch.Size([502, 4096])
torch.Size([502, 21])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [1, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [113]:
class CustomNN(nn.Module):
    def __init__(self, bits):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, bits),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [114]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float().clamp(max = 1)
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [ ]:
# Define the grid

def DPSH(device: torch.device, X_train: np.ndarray, y_train: np.ndarray, bit: int, num_epoch: int, batch_size: int, lr: int, weight_decay: int, eta: int):

    n_classes = y_train.shape[1]
    train_loader, train_size = CreateDataset(X = X_train, y = y_train, batch_size = 128, train = 1)





    customLoss = DPSHLoss(train_size, n_classes, bit)

    # Initialize model and optimizer
    model = CustomNN(bits = bit).to(device)
    optimizer = optim.Adam(model.parameters(), lr, weight_decay)

    # Train the model
    for epoch in range(num_epoch):  # Example epoch count
        #current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))
        #print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
        #"DPSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")
        model.train()
        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = model(image)

            loss = customLoss(u, label.float(), ind, eta)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / (train_size / batch_size)
        #print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))
    return model
    

In [ ]:
DPSH12Nus = DPSH(device, 5002, 21, 12, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
DPSH24Nus = DPSH(device, 5002, 21, 24, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
DPSH32Nus = DPSH(device, 5002, 21, 32, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
DPSH48Nus = DPSH(device, 5002, 21, 48, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])

In [ ]:
DPSH48Cifar = {'eta': 0.05, 'learning_rate': 1e-05, 'weight_decay': 1e-05}
DPSH32Cifar = {'eta': 0.1, 'learning_rate': 1e-05, 'weight_decay': 1e-05}
DPSH24Cifar = {'eta': 0.1, 'learning_rate': 1e-05, 'weight_decay': 0.0001}
DPSH12Cifar = {'eta': 0.01, 'learning_rate': 1e-05, 'weight_decay': 1e-06}
DPSH12Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH24Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-05}
DPSH32Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH48Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH12Nus = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-05}
DPSH24Nus = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-05}
DPSH32Nus = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 0.0001}
DPSH48Nus = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-06}

{'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-06}

: 

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset

# Dummy dataset
class DummyDataset(Dataset):
    def __init__(self, size):
        self.data = torch.arange(size)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

# Create dataset and dataloader
dataset = DummyDataset(size=20)
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Collect all batches and verify order
all_batches = []
for batch in dataloader:
    all_batches.append(batch)

# Concatenate all batches
concatenated = torch.cat(all_batches)

# Check if order is preserved
original = torch.arange(len(dataset))
order_preserved = torch.equal(original, concatenated)

print("Concatenated Tensor:", concatenated)
print("Order Preserved:", order_preserved)


Concatenated Tensor: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])
Order Preserved: True


In [2]:
import torch
from torch.utils.data import DataLoader

# Create a tensor
data = torch.arange(20)

# Wrap the tensor with a DataLoader
batch_size = 4
dataloader = DataLoader(data, batch_size=batch_size, shuffle=False)

# Collect all batches and verify order
all_batches = []
for batch in dataloader:
    all_batches.append(batch)

# Concatenate all batches
concatenated = torch.cat(all_batches)

# Check if order is preserved
original = torch.arange(len(data))
order_preserved = torch.equal(original, concatenated)

print("Concatenated Tensor:", concatenated)
print("Order Preserved:", order_preserved)


Concatenated Tensor: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])
Order Preserved: True
